In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
from time import sleep
import joblib

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Pages retrieval

In [2]:
import requests

def url_checker(url):
    try:
        get = requests.get(url)  # get url
        if get.status_code == 200:  # request succeeds 
            return 1, get.content
        else:
            return 0, ''

    #Exception
    except requests.exceptions.RequestException as e:
        # print URL with Errs
        raise SystemExit(f"{url}: is Not reachable \nErr: {e}")

In [3]:
# results = {}
# search_range = np.arange(188500 - 1000, 188500 + 1000)
# progress_bar = tqdm(search_range)
# num_finded_pages = 0

# for page_id in progress_bar:
#     cur_http = f"https://old.gu.spb.ru/{page_id}/eservice/"
#     status, text = url_checker(cur_http)
    
#     # dump page
#     if status:
#         results[cur_http] = text
#         num_finded_pages += 1
        
#     progress_bar.set_description(f"Finded {num_finded_pages}, cur ID: {page_id}")    
#     sleep(0.1)
# #     print(page_id)

Finded 585, cur ID: 189499: 100%|███████████| 2000/2000 [11:38<00:00,  2.86it/s]


In [4]:
# joblib.dump(results, 'parsing_results.joblib')

['parsing_results.joblib']

## Pages parsing

In [2]:
results = joblib.load('parsing_results.joblib')

In [27]:
results['https://old.gu.spb.ru/188411/eservice/']

In [101]:
tags = ["Общее описание",
        "Результат предоставления",
        "Срок предоставления",
        "Заявители",
        "Порядок действий",
        "Документы"
       ]

In [168]:
my_str = "и ребенка\xa0—\xa0при родах вне медицинской"
print(my_str)

и ребенка — при родах вне медицинской


In [156]:
import re

In [172]:
regex = re.compile(r'[\n\r\t]')
regex2 = re.compile(r'[\xa0]')
regex.sub('', my_str)
regex2.sub(' ', my_str)

'и ребенка — при родах вне медицинской'

In [177]:
def parse_url(bytes_html, relevant_tags=tags):
    regex = re.compile(r'[\n\r\t\xa0]')
    
    soup = bs(bytes_html, 'html.parser')
    
    h1_tag = soup.find('h1').find('span').text.strip()
    h1_tag = regex.sub('', h1_tag)
    all_templates = []
    
    # parse h2
    all_h2 = soup.find_all('h2')
    for cur_h2 in all_h2:
        if cur_h2.text in relevant_tags:
            text_between_h2 = []
            cur_tag = cur_h2.find_next_sibling()
            while cur_tag:
                text_between_h2.append(str(cur_tag))
                cur_tag = cur_tag.find_next_sibling()

            text_between_h2 = '\n'.join(text_between_h2) 
            
            # parse h3
            h2_soup = bs(text_between_h2, 'html.parser')
            all_h3 = h2_soup.find_all('h3')
            for cur_h3 in all_h3:
                text_between_h3 = []
                cur_tag = cur_h3.find_next_sibling()
                while cur_tag:
                    if cur_tag.name == 'h3':
                        joined_text = "\n".join(text_between_h3)
                        template = "\n".join([f"Услуга: {h1_tag}",
                                              f"Заголовок: {regex.sub('', cur_h2.text)}",
                                              f"Подраздел: {regex.sub('', cur_h3.text)}",
                                              f"Текст: {regex.sub('', joined_text)}"
                                             ])
                        all_templates.append(template)
                        text_between_h3 = []
                        break
                    else:
                        text_between_h3.append(cur_tag.text)
                        cur_tag = cur_tag.find_next_sibling()
                if text_between_h3:
                    joined_text = "\n".join(text_between_h3)
                    template = "\n".join([f"Услуга: {h1_tag}",
                                          f"Заголовок: {regex.sub('', cur_h2.text)}",
                                          f"Подраздел: {regex.sub('', cur_h3.text)}",
                                          f"Текст: {regex.sub('', joined_text)}"
                                         ])
                    all_templates.append(template)                    
            if not all_h3:
                template = "\n".join([f"Услуга: {h1_tag}",
                                      f"Заголовок: {regex.sub('', cur_h2.text)}",
                                      f"Текст: {regex.sub('', h2_soup.text)}"
                                     ])
                all_templates.append(template)
                
    return all_templates

In [178]:
a = ['1', '2']
a.extend(['12', '2'])
a

['1', '2', '12', '2']

In [129]:
chunks = []
for http, document in tqdm(results.items()):
    chunks.extend(parse_url(document, relevant_tags=tags))
    break

100%|█████████████████████████████████████████| 585/585 [00:12<00:00, 47.84it/s]


In [181]:
chunks = []
for http, document in tqdm(results.items()):
    chunks.extend(parse_url(document, relevant_tags=tags))

100%|█████████████████████████████████████████| 585/585 [00:12<00:00, 47.72it/s]


In [136]:
list(results.keys())[-1]

'https://old.gu.spb.ru/189497/eservice/'

In [137]:
# import csv

# # field names
# fields = ['Context']

# with open('mfc_context_chunks.csv', 'w') as f:
#     # using csv.writer method from CSV package
#     write = csv.writer(f)
#     write.writerow(fields)
#     write.writerows(chunks)

In [184]:
df = pd.DataFrame(chunks, columns=['Context'])

In [185]:
df.to_csv('mfc_context_chunks_cleaned.csv')